In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/06 23:11:03 WARN Utils: Your hostname, asaad resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp61s0)
24/03/06 23:11:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/06 23:11:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Q2
pyspark.__version__

'3.3.2'

In [6]:
!head fhv_tripdata_2019-10.csv

dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264,264,,B00009
B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264,264,,B00013
B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264,264,,B00014
B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264,264,,B00014
B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264,264,,B00014
B00021         ,2019-10-01 00:00:48,2019-10-01 00:07:12,129,129,,B00021         
B00021         ,2019-10-01 00:47:23,2019-10-01 00:53:25,57,57,,B00021         
B00021         ,2019-10-01 00:10:06,2019-10-01 00:19:50,173,173,,B00021         
B00021         ,2019-10-01 00:51:37,2019-10-01 01:06:14,226,226,,B00021         


In [8]:
from pyspark.sql import types


In [45]:
# Define the schema with correct column names

schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PUlocationID', types.IntegerType(), True),
    types.StructField('DOlocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [46]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [68]:
df.show(5)
df = df.repartition(6)


+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01437|2019-10-01 13:21:10|2019-10-01 13:25:44|         264|         134|   null|                B02765|
|              B02243|2019-10-01 12:38:00|2019-10-01 14:02:00|         202|          11|   null|                LX-176|
|              B02881|2019-10-03 09:17:55|2019-10-03 09:33:00|         226|          82|   null|                B02881|
|              B00987|2019-10-03 14:11:00|2019-10-03 14:55:00|          89|         227|   null|                B00987|
|              B01601|2019-10-01 15:58:00|2019-10-01 16:12:00|         264|         264|   null|                B01601|
+--------------------+------------------

In [51]:
df.write.parquet('fhvhv/2019/10/')

In [52]:
df = spark.read.parquet('fhvhv/2019/10/')

In [53]:
df.printSchema()


root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [54]:
from pyspark.sql import functions as F
# import col function
from pyspark.sql.functions import col

In [58]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [66]:
df \
    .withColumn('duration', (df.dropOff_datetime.cast('long') - df.pickup_datetime.cast('long')) / 3600) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(1) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-10-28|     631152.5|
+-----------+-------------+



In [63]:
df_zones = spark.read \
.option("header", "true") \
.csv('taxi_zone_lookup.csv')

In [67]:
df_zones.columns
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [65]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [69]:
# Join the two DataFrames on PUlocationID and LocationID
joined_df = df.join(df_zones, df.PUlocationID == df_zones.LocationID)

# Group by the pickup zone and count the occurrences
pickup_zone_counts = joined_df.groupBy('Zone').count()

# Find the least frequent pickup location zone
least_frequent_zone = pickup_zone_counts.orderBy('count').limit(1)

# Show the result
least_frequent_zone.show()

+-----------+-----+
|       Zone|count|
+-----------+-----+
|Jamaica Bay|    1|
+-----------+-----+

